<a href="https://colab.research.google.com/github/k2moon/bigbungi/blob/main/test/bigbungi_work02_classifier_01_ans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 ▶ 제2유형  분류 모델 수행하기
---


## 캐글 타이타닉 생존 여부 예측

다음은 Titanic - Machine Learning from Disaster 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 생존 여부를 예측하고 해당 결과를 csv파일로 제출하시오.
(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

* 결과 제출 양식 : PassengerId, Survived

- DataSet: https://www.kaggle.com/competitions/titanic
- DataSet x_test : https://tinyurl.com/yew5uc73
- DataSet x_train : https://tinyurl.com/37jrwyay
- DataSet y_train : https://tinyurl.com/3r5hmh84

In [62]:
# 데이터 가져오기 read_csv()
import pandas as pd
import numpy as np

x_test = pd.read_csv("https://tinyurl.com/yew5uc73", encoding='CP949')
x_train = pd.read_csv("https://tinyurl.com/37jrwyay", encoding='CP949')
y_train = pd.read_csv("https://tinyurl.com/3r5hmh84")

In [68]:
# 값 확인
x_train.isnull().sum()

티켓등급    0
성별      0
운임요금    0
선착장     0
가족수     0
dtype: int64

In [63]:
# ------- 데이터 확인 및 EDA ---------

# 모든 컬럼값 확인 T
print(x_train.head(1).T)

# 행/열 확인 shape (생략가능)
print(x_train.shape)

# 요약정보 확인 info()
## 데이터타입, 결측치 등 확인, object 범주형
### object 범주형 인코딩 확인
print(x_train.info())

# 기초통계량 확인 describe()
## count, mean,std,min, 25%, 50%, 75%, max
## 데이터 스케일링 확인, 회귀, 비tree 계열
print(x_train.describe())
#print(X_train.describe().loc['min',:])

# 타깃값 확인 unique
## 1차원인지도 확인 (y,)
print(y_train['Survived'].value_counts())
y_train = y_train['Survived']
print(y_train.unique())

# EDA
data = pd.concat([x_train, y_train], axis=1)

## 성별 칼럼에 따른 생존 Survived 평균값
print(data.groupby(['성별'])['Survived'].mean())

## 티켓등급 칼럼에 따른 생존 Survived 평균값
print(data.groupby(['티켓등급'])['Survived'].mean())

## 선착장 칼럼에 따른 생존 Survived 평균값
print(data.groupby(['선착장'])['Survived'].mean())

# ------- 데이터 전처리 preprocessing ---------
## train, test 모두 처리 해야 함

# 불필요한 컬럭 삭제 drop(columns=[])
## PK 역할 컬럼 등 삭제
x_train = x_train.drop(columns = ['PassengerId'])
y_train = y_train.drop(columns = ['PassengerId'])

# 제출용 id 삭제전 저장
x_test_id = x_test['PassengerId']
x_test = x_test.drop(columns = ['PassengerId'])

print(x_train.isnull().sum()) # 티켓번호,객실번호,승객이름 삭제 이유는?
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])
x_train = x_train.drop(columns=['객실번호'])
x_test = x_test.drop(columns=['객실번호'])
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])
print(x_train.isnull().sum())


# 결측치 처리하기 fillna()
## 임의로 결측치 삭제하지 말 것
## 평균, 중앙값, 상황에 따른 값
## 나이, 선착장

## 나이 상관관계 확인 - 상관이 없음 - 삭제
print(data[['나이','Survived']].corr())
x_train = x_train.drop(columns=['나이'])
x_test = x_test.drop(columns=['나이'])
## 나이 상황에 따른 값으로 처리 해도 됨 - 평균 등

## 선착장 2건 상황에 따른 값으로 처리 - 평균, 최빈값
v = x_train['선착장'].value_counts()

x_train['선착장'] = x_train['선착장'].fillna(v)
x_test['선착장'] = x_test['선착장'].fillna(v)


# 범주형 변수 인코딩 objedt형
## 라벨 인코딩 LabelEncoder, 원핫인코딩  OneHotEncoder
## 라벨 인코딩(Tree 계열의 분류 알고리즘에 사용)
## 성별, 선착장
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
print(x_train['성별'].unique())
x_train['성별'] = encoder.fit_transform(x_train['성별'])
x_test['성별'] = encoder.fit_transform(x_test['성별'])
print(x_train['성별'].unique())
print(x_test['성별'].unique())
x_train['선착장'] = encoder.fit_transform(x_train['선착장'])
x_test['선착장'] = encoder.fit_transform(x_test['선착장'])
print(x_test['선착장'].unique())

# 파생변수 생성
## 필요시 생성
## 유사 칼럼 : 부모자식수, 형제자매배우자수  => 가족수 만들기
x_train['가족수'] = x_train['부모자식수'] + x_train['형제자매배우자수']
x_train = x_train.drop(columns=['부모자식수', '형제자매배우자수'])

x_test['가족수'] = x_test['부모자식수'] + x_test['형제자매배우자수']
x_test = x_test.drop(columns=['부모자식수', '형제자매배우자수'])

# 데이터 스케일링
## 연속형 변수의 최대, 최소 분포차가 클때
## MinMaxScaler, StandardScaler, RobustScaler
## test는 train용으로 transform()
## Tree 계열 필수는 아님

print(x_train.describe().T)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
print(x_train.describe().T)

# 상관관계 확인 corr()
## 필요 시 확인

# 전처리 확인 info()
print(x_train.info())
print(x_test.info())

# ------- 머신러닝 machine learning ---------

# 평가용 데이터 분리 model_selection
## stratify, stes_size, y값 1차원 확인
print(y_train.value_counts()/len(y_train))
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
print(Y_train.value_counts()/len(Y_train))

# 데이터 학습
## fit, predict, predict_proba
## 분류 XXXClassifier, LogisticRegression
## 회귀(예측) XXXRegressor(XXXRegression)
## 공통 ensemble
## RandomForestXXX : n_estimators (default = 100), max_depth, criterion
## criterion : 분할 품질을 측정하는 기능 ()
### RandomForestClassifier: criterion{“gini”, “entropy”, “log_loss”}, default=”gini”
### RandomForestRegressor: criterion{“mse”, “mae”}, / v1.2 criterion{“squared_error”, “absolute_error”}, default=”squared_error”
## GradientBoostingXXX : n_estimators, learning_rate(default =  0.1)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


#model = DecisionTreeClassifier(random_state=42) 
#model = LogisticRegression(random_state=42) 
model = RandomForestClassifier(random_state=42)  
#model = GradientBoostingClassifier() 

model.fit(X_train, Y_train)
pred = model.predict(X_test)


# 데이터 평가 metrics
## 회귀 : MAE, MSE, RMSE, R^2, RMSE는 np.sqrt(MSE값)
## 분류 : ROC_AUC, Accuracy(정확도), Precision(정밀도), Recall(재현율)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(roc_auc_score(Y_test, pred))


# ------- 답안 제출 ---------

# 전체학습데이터로 다시 학습
model.fit(x_train, y_train)

# 제출용 예측
pred = model.predict(x_test)


# 답안 제출 참고
# #아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
#pd.DataFrame({'cust_id': x_test_data.cust_id, 'gender': pred}).to_csv('424242.csv', index=False)

# 제출 최종 확인
#print(pd.read_csv("data/424242.csv"))

                                   0
PassengerId                        1
티켓등급                               3
승객이름         Braund, Mr. Owen Harris
성별                              male
나이                              22.0
형제자매배우자수                           1
부모자식수                              0
티켓번호                       A/5 21171
운임요금                            7.25
객실번호                             NaN
선착장                                S
(891, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891

ValueError: ignored